In [ ]:
# /content/complaints.csv

In [ ]:
# Data  --- raw /processfile/multiple /sources data

# models --> different models are stored here( model1 , model2, 3)
# experiment tracking :- commenting for the code

# notebooks--->

# output --->

# reports --->

# src/core/app :-
# backend code/ api/ forntend


In [ ]:
# step 1 :-
# Understand problem (in a bussiness context)

# cfbc:
# The Consumer Financial Protection Bureau is an independent agency of the United States government
# responsible for consumer protection in the financial sector

# step 2:-  Understand problem (in technical perspective/nlp point of view)

# it is a classification problem ---> multiclass classification problem
# text classification


# # factors to consider
# how much data we have

# format of data

# time duration
# computational resources
# how much control over output needed : explainability


In [ ]:
#  NLP Project lifecycle
# 1. Data gathering

#  client database: MongoDB
# manager was given the access to the databse he downloaded the data to the server or s3 bucket(sagemaker)


# import libraries

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/input/complaints-csv/complaints.csv')

EDA :- Exploratory data analysis

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df  = df[['Product','Consumer complaint narrative']]

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.isna().sum()/ len(df)*100

In [ ]:
# # potential solutions

# Data Augmentation for same target (product) column

# data generation using llm's using (chat gpt)

# drop na values

In [ ]:
#  we choose drop na
# even after dropping we will have row, that will be enough
# around 54% data is null , if we generate all data model might overfit to generated data

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Product'].nunique()

In [ ]:
df['Product'].unique()

In [ ]:
# we have 18 classes some of them are closely related we can merge them

# after discussing with client we decided to merge some of them


In [ ]:
# 'Vehicle loan or lease',
# 'Credit reporting, credit repair services, or other personal consumer reports',
# 'Credit card or prepaid card',
# 'Money transfer, virtual currency, or money service', 'Mortgage',
# 'Payday loan, title loan, or personal loan', 'Debt collection',
# 'Checking or savings account', 'Student loan', 'Consumer Loan',
# 'Money transfers', 'Credit card', 'Bank account or service',
# 'Credit reporting', 'Prepaid card', 'Payday loan',
# 'Other financial service', 'Virtual currency

In [ ]:
# loan -->'Vehicle loan or lease', 'Student loan'
#           'Consumer Loan', 'Payday loan, 'Payday loan, title loan, or personal loan'


# credit repot --> 'Credit reporting, credit repair services, or other personal consumer reports',
#                   'Credit reporting'

# card  --->  'Prepaid card', 'Credit card or prepaid card', 'Credit card'

# money transfer --> 'Money transfer, virtual currency, or money service',
#                     'Money transfers', 'Virtual currency'

# account -->  'Checking or savings account', 'Bank account or service'

# Mortgage

# 'Debt collection'

# 'Other financial service'

In [ ]:
class_dict =  {

'Vehicle loan or lease' : 'loan',
'Credit reporting, credit repair services, or other personal consumer reports' : 'credit_report',
'Credit card or prepaid card' : 'card' ,
'Money transfer, virtual currency, or money service' : 'money_transfer',
'Mortgage' : 'Mortgage',
'Payday loan, title loan, or personal loan' : 'loan' ,
'Debt collection' : 'Debt collection',
'Checking or savings account' : 'account',
'Student loan' : 'loan',
'Consumer Loan' : 'loan',
'Money transfers' : 'money_transfer',
'Credit card' : 'card',
'Bank account or service' : 'account',
'Credit reporting' : 'credit_report',
'Prepaid card' : 'card',
'Payday loan' : 'loan',
'Other financial service' : 'others',
'Virtual currency' : 'money_transfer'
}

In [ ]:
df.replace({'Product' : class_dict}, inplace=True)

In [ ]:
df

In [ ]:
# df['Product'].replace(class_dict)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df['Product'].unique()

In [ ]:
df['Product'].nunique()

In [ ]:
df['Product'].value_counts()

In [ ]:
df['Product'].value_counts()/len(df)*100

In [ ]:
#  dataset is imbalanced

# Smote
#  oversampling, undersampling
# class weights

#  proper evaluation metrics


In [ ]:
#  Text cleaning

df

In [ ]:
# in NLP three important compenents to improve performance of model
# 1. text preprocessing
# 2. vectorization
# 3. model building

In [ ]:
df['Consumer complaint narrative'].iloc[200]

In [ ]:
# 1. Remove XXXX
# 2. lower
# 3. punctuations
# 4. remove digits
# 5. new line characteres


In [ ]:
import re

In [ ]:
def clean_text(text):
  text = text.lower()  # lowercase

  text = re.sub(r'[^\w\s]', '', text)   # removing punctuation and digits
  text = re.sub(r'[x]{2,}',"",text)  # removing more than one occereance of x

  return text



In [ ]:
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(clean_text)

In [ ]:
length = []

for i in df['Consumer complaint narrative']:
  length.append(len(i))

length.sort()

length[100]

In [ ]:
#  very short length review can be removed


In [ ]:
sum(length)/len(length)

In [ ]:
!pip install keras-preprocessing

In [ ]:
# Tokenize
!pip install tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=30000, lower = True)

In [ ]:
tokenizer.fit_on_texts(df['Consumer complaint narrative'])

In [ ]:
word_index = tokenizer.word_index   # dictinory

In [ ]:
word_index['check']

In [ ]:
value_to_find = 145

for key, value in word_index.items():
  if value == value_to_find:
    print(key)


In [ ]:
value_to_find = 1

for key, value in word_index.items():
  if value == value_to_find:
    print(key)

In [ ]:
# remove stop words
# remove more then one occurance of space

In [ ]:
#  vectorization : converting text to numbers

In [ ]:
X = tokenizer.texts_to_sequences(df['Consumer complaint narrative'].values)

In [ ]:
df['Product']

In [ ]:
Y = pd.get_dummies(df['Product']).values

In [ ]:
len(X[0])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
x = pad_sequences(X, maxlen = 1000)


In [ ]:
# x[0]

In [ ]:
len(x)

In [ ]:
len(Y)

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(x, Y, test_size=0.05, random_state=42)

In [ ]:
#  model

In [ ]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, SpatialDropout1D, Bidirectional

In [ ]:
#Model


model=Sequential()
model.add(Embedding(50000,100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
class_labels = np.unique(df['Product'])

from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight ='balanced',
classes=np.unique(df['Product']),y = df['Product'])

# class_weights.sort()
class_weights_dict = dict(zip(class_labels, class_weights))

In [ ]:
history = model.fit(x_train,
                    y_train,
                    epochs=7,
                    batch_size=2064,
                    validation_split=0.1,
                    class_weight=class_weights_dict,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
model.summary()

In [ ]:
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
pred = model.predict(x_test)

In [ ]:
# confusion matrix in sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
y_pred=np.argmax(pred, axis=1)

In [ ]:
y_test_=np.argmax(y_test, axis=1)

In [ ]:
len(y_test_)

In [ ]:
# confusion matrix
matrix = confusion_matrix(y_test_,y_pred)

In [ ]:
matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(matrix,
            annot=True,)
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)

In [ ]:
# 0 : Debt collection
# 1 : Mortgage
# 2 : account
# 3 : card
# 4 : credit_report
# 5 : loan
# 6 : money_transfer
# 7 : others

In [ ]:
# 4 getting misclasiify by --> 0,1,3,5
# 5 getting misclasiify by --> 0,4
# 3 getting misclasiify by --> 0,2, 4
# 2 getting misclasiify by --> 3

In [ ]:
# Debt collection	Mortgage	account	card	credit_report	loan	money_transfer	others

In [ ]:
#Error Analysis
# type 1 : FP

# type 2: FN

# accuracy

# precision  : minimun type 1 error  --> Spam classifiaction

# recall : minimun type 1 error --> diesese prediction

In [ ]:
print(classification_report(y_test_, y_pred))

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()

In [ ]:
df['Product'].value_counts()

In [ ]:
df.head()

In [ ]:
df['Consumer complaint narrative'][10]

In [ ]:
new_complaint = ['''i contacted ally on friday after falling behind on payments due to being out of work for a short period of time due to an illness i chated with a representative after logging into my account regarding my opitions to ensure i protect my credit and bring my account current

she advised me that before an extenstion could be done i had to make a payment in the amount of i reviewed my finances as i am playing catch up on all my bills and made this payment on monday this rep advised me once this payment posts to my account to contact ally back for an extention or to have a payment deffered to the end of my loan

with this in mind i contacted ally again today and chatted with i explained all of the above and the information i was provided when i chatted with the rep last week she asked several questions and advised me that a one or two month extension deffered payment could be done however partial payment is needed what she advised me or there abouts would be due within days from me accepting the agreement and then the remaining bal of or there abouts would be due in in my payments of per month would resume

if this was the case i should have just been offered this when i just made my payment so that i could catch up on my bills

this company was working with in new jersey which has since closed most likely due to illegal practices they changed my loan company to this company after i had signed paperwork for another kill you with interest rates and has never once considered refiancing my vechile for a lower interest rate due to the age of the vechile other companies will not take it and they do not work with you ''']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=1000)
pred = model.predict(padded)
labels = ['credit_report','debt_collection','mortgage','card','savings_account', 'loan', 'money_transfer', 'vehicle_loan', 'others']
print(pred, labels[np.argmax(pred)])

In [ ]:
# Y = pd.get_dummies(df['Product'])

In [ ]:
df.head()

In [ ]:
new_dict = {'others':'credit_report'}
df.replace({'Product' : new_dict}, inplace=True)

In [ ]:
df['Product'].value_counts()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Consumer complaint narrative'],
                                                    df['Product'],
                                                    test_size=0.05,
                                                    random_state=42)

In [ ]:
X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)

In [ ]:
X_train

In [ ]:
# Train the Word2Vec model
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

In [ ]:
vocab_size = 30000

In [ ]:
# Create a weight matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Pad the sequences to a fixed length
max_length = 1000
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

In [ ]:
#Define Model
model = Sequential()
model.add(Embedding(30000, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))

model.add(SpatialDropout1D(0.2))
model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])